In [89]:
!pip install pymystem3

!pip install pymorphy2


Считываем книгу

In [90]:
f = open('book.txt', 'r',  encoding="utf-8")
text = f.read()
f.close()

Обрабатываем при помощи MyStem и сохраняем

In [96]:
%%time
from pymystem3 import Mystem
from pprint import pprint
import json
m = Mystem()
ana = m.analyze(text)
with open('mystem.json', 'w', encoding='utf-8') as f:
    json.dump(ana, f, ensure_ascii = False, indent = 4)

Токенизируем при помощи nltk, удаляем знаки препинания

In [91]:
import nltk
from nltk.tokenize import word_tokenize
words = [w.lower() for w in word_tokenize(text) if w.isalpha()]

обрабатываем при помощи pymorphy и сохраняем

In [92]:
%%time
import json
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
l = []
an = []
lemmas = []
for word in words:
    s = []
    anali = morph.parse(word)
    an.append(anali[0])
    lemma = anali[0].normal_form
    lemmas.append (lemma)
    pos = anali[0].tag.POS
    s.append(lemma)
    s.append(pos)
    l.append(s)
with open('pymorphy.json', 'w', encoding='utf-8') as f:
    json.dump(l, f, ensure_ascii = False)

обрабатываем при помощи pymorphy, создаем частотный словарь для частей речи, находим долю для каждой части речи

In [93]:
d = {}
for word in an:
    if word.tag.POS in d:
        d[word.tag.POS] += 1
    else:
        d[word.tag.POS] = 1
for pos in d:
    print(pos, ' - ', d[pos]/len(words))

Делаем частотный словарь наречий и глаголов, сортируем по частотности, выводим результат

In [94]:
import operator
v = {}
a = {}
for word in an:
    if word.tag.POS == 'VERB':
        if word.normal_form in v:
            v[word.normal_form] += 1
        else:
            v[word.normal_form] = 1
    if word.tag.POS == 'ADVB':
        if word.normal_form in v:
            a[word.normal_form] += 1
        else:
            a[word.normal_form] = 1
print('Глаголы:')
sorted_v = sorted(v.items(), key=operator.itemgetter(1))
for i in range(20):
    k = (len(sorted_v) - i - 1)
    print(sorted_v[k][0])
print()
print('Наречия:')
sorted_a = sorted(a.items(), key=operator.itemgetter(1))
for i in range(20):
    k = (len(sorted_a) - i - 1)
    print(sorted_a[k][0])

Находим и составляем все биграммы и триграммы, делаем частотный словарь, сортируем и выводим самые частотные

In [95]:
u = []
r = []
b = {}
t = {}
print('Биграммы:')
bigrm = nltk.bigrams(lemmas)
for i in bigrm:
    bi = i[0] + ' ' + i[1]
    u.append(bi)
for words in u:
    if words in b:
        b[words] += 1
    else:
        b[words] = 1
sorted_b = sorted(b.items(), key=operator.itemgetter(1))
for i in range(25):
    k = (len(sorted_b) - i - 1)
    print(sorted_b[k][0])
print()
print('Триграммы:')
trgrm = nltk.trigrams(lemmas)
for i in trgrm:
    tr = i[0] + ' ' + i[1] + ' ' + i[2]
    r.append(tr)
for words in r:
    if words in t:
        t[words] += 1
    else:
        t[words] = 1
sorted_t = sorted(t.items(), key=operator.itemgetter(1))
for i in range(25):
    k = (len(sorted_t) - i - 1)
    print(sorted_t[k][0])